# Workshop Microsoft - Sostenibilidad
## Ejercicios
A continuación haremos un ejercicio con codecarbon

In [ ]:
%pip install transformers==4.48 accelerate sentencepiece tokenizer codecarbon -q 
%pip install dash dash-bootstrap-components python-dotenv -q
%pip install flash_attn -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Posiblemente, sea necesario reiniciar el kernel

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import textwrap
from codecarbon import EmissionsTracker
from codecarbon import track_emissions
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
from dotenv import load_dotenv

#os.environ["TOKENIZERS_PARALLELISM"] = "false"


load_dotenv()

False

In [ ]:
# parametros de configuración
model_name = "microsoft/Phi-3.5-mini-instruct"
project_name="worshop-ia-sostenible-exercise"

#TODO CAMBIAR RUTA DEL FICHERO
#output_dir_emissions_track="/lakehouse/default/Files"
output_dir_emissions_track="/home/azureuser/cloudfiles/code/Users/paul.vanb/worshop-ia-sostenible/outputs"
output_file_emissions_track=f"{project_name}_emissions.csv"

output_filename = os.path.join(output_dir_emissions_track,output_file_emissions_track )
device_map="auto"
cloud_region="westus2"

In [ ]:

def wrap_text(text, width=100):
    """
    Envuelve el texto dado para que cada línea no supere el ancho especificado.
    
    Args:
        text (str): El texto de entrada.
        width (int): El ancho máximo de cada línea (por defecto, 100 caracteres).
    
    Returns:
        str: Texto con saltos de línea insertados para cumplir con el ancho especificado.
    """
    lines = text.split('\n')  # Divide el texto en líneas basadas en saltos de línea existentes.
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]  # Envuelve cada línea individualmente.
    wrapped_text = '\n'.join(wrapped_lines)  # Une las líneas envueltas con saltos de línea.
    return wrapped_text

def generate(pipe, input_text, system_prompt=""):
    """
    Genera una respuesta basada en el texto de entrada utilizando un modelo de IA.
    
    Args:
        input_text (str): Texto de entrada del usuario.
        system_prompt (str): Mensaje de sistema opcional para guiar la respuesta del modelo.
        max_length (int): Longitud máxima de la respuesta generada (por defecto, 1024 caracteres).
    
    Returns:
        str: Texto generado y formateado con saltos de línea adecuados.
    """
    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }
    # Si no se proporciona un mensaje del sistema, se usa un mensaje por defecto.
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    
    # Construcción del historial de mensajes con el mensaje del sistema y la entrada del usuario.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text},
    ]
    
    # Generación de la respuesta del modelo.
    output = pipe(messages, **generation_args)
    text = output[0]['generated_text']  # Obtiene el texto generado.
    
    # Envuelve el texto generado para cumplir con el ancho especificado.
    wrapped_text = wrap_text(text)
    return wrapped_text

@track_emissions(project_name= project_name, output_dir=output_dir_emissions_track,
                 cloud_provider="azure", cloud_region= cloud_region, pue=1.19, output_file=output_file_emissions_track,
                 allow_multiple_runs=True)
def answer_to_different_messages(pipe,messages):
    """
    Procesa una lista de mensajes y genera respuestas mientras realiza un seguimiento de las emisiones.
    
    Args:
        messages (list): Lista de mensajes de entrada.
    
    Returns:
        list: Lista de respuestas generadas junto con los mensajes de entrada y sus identificadores.
    """
    outputs = []  # Lista para almacenar los resultados.
    system_prompt = "Eres un asistente de Microsoft experto en sostenibilidad llamado SostAI"  # Mensaje del sistema.
    
    # Itera sobre cada mensaje y genera una respuesta.
    for mid, message in enumerate(messages):
        output = generate(pipe, message, system_prompt=system_prompt)  # Genera la respuesta.
        outputs.append([mid, message, output])  # Almacena el identificador, mensaje y respuesta.
    
    return outputs  # Devuelve la lista de respuestas generadas.

###############################################
#################### PLOTS ####################
###############################################

def plot_stacked_power(df):
    """
    Genera un gráfico de barras apiladas para visualizar el consumo de potencia (CPU, GPU, RAM) a lo largo del tiempo.
    
    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas 'timestamp', 'cpu_power', 'gpu_power' y 'ram_power'.
    """
    df_sorted = df.sort_values(by='timestamp')  # Ordenar por timestamp
    
    plt.figure(figsize=(12, 6))
    plt.bar(df_sorted['timestamp'], df_sorted['cpu_power'], label='CPU Power', color='blue')
    plt.bar(df_sorted['timestamp'], df_sorted['gpu_power'], bottom=df_sorted['cpu_power'], label='GPU Power', color='red')
    plt.bar(df_sorted['timestamp'], df_sorted['ram_power'], bottom=df_sorted['cpu_power'] + df_sorted['gpu_power'], label='RAM Power', color='green')
    
    plt.xlabel('Timestamp')
    plt.ylabel('Power Consumption (W)')
    plt.title('Stacked Power Consumption Over Time')
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

def plot_stacked_energy(df):
    """
    Genera un gráfico de barras apiladas para visualizar el consumo de energía (CPU, GPU, RAM) a lo largo del tiempo.
    """
    df_sorted = df.sort_values(by='timestamp')
    
    plt.figure(figsize=(12, 6))
    plt.bar(df_sorted['timestamp'], df_sorted['cpu_energy'], label='CPU Energy', color='blue')
    plt.bar(df_sorted['timestamp'], df_sorted['gpu_energy'], bottom=df_sorted['cpu_energy'], label='GPU Energy', color='red')
    plt.bar(df_sorted['timestamp'], df_sorted['ram_energy'], bottom=df_sorted['cpu_energy'] + df_sorted['gpu_energy'], label='RAM Energy', color='green')
    
    # Agregar Total Energy Consumed como una barra al lado de cada apilamiento
    bar_width = 0.28  # Definir el ancho de las barras
    x_indexes = range(len(df_sorted['timestamp']))
    plt.bar([x + bar_width for x in x_indexes], df_sorted['energy_consumed'], label='Total Energy Consumed', color='purple', alpha=0.7, width=bar_width)
    
    plt.xlabel('Timestamp')
    plt.ylabel('Energy Consumption (kWh)')
    plt.title('Stacked Energy Consumption Over Time')
    plt.xticks([x + bar_width / 2 for x in x_indexes], df_sorted['timestamp'], rotation=45)
    plt.legend()
    plt.show()

[codecarbon INFO @ 17:38:09] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.092193603515625 W
[codecarbon INFO @ 17:38:09] Energy consumed for all CPUs : 0.000407 kWh. Total CPU Power : 97.5 W
[codecarbon INFO @ 17:38:09] 0.000419 kWh of electricity used since the beginning.


# Carga del Modelo
Usaremos la librería transformers de Hugginface para cargar el modelo de microsoft Phi-3.5 mini instruído.


In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map= device_map,
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.79it/s]


# Ejercicio 1
Vamos a observar las diferencias en coste energético entre dos regiones diferentes para eso vamos a ejecutar con una lista mensajes definida, la función answer_to_different_messages para el país de brasil (country_code = BRA) y otra para la de Irlanda (country_code = IRL).
Para luego diferenciarlas en el output, asigna a la primera project_name "phi-emissions-bra" y a la segunda "phi-emissions-irl".

Para hacer esto, usaremos OfflineEmissionTracker

In [ ]:
messages = ["cuanto cuesta ser sostenible?", "Qué debo hacer para ser sostenible?", "Qué son los algoritmos verdes?","Cuesta mucho ser eficiente?",
            "Qué es la eficiencia energética?"]
print(len(messages))

In [ ]:
from codecarbon import OfflineEmissionsTracker


# Ejercicio 2 (opcional)
Ahora vamos a observar las diferencias en coste energético entre cpu y gpu, para eso vamos a ejecutar con una lista de 20 mensajes a tu elección, la función answer_to_different_messages con gpu y luego cpu.
Para luego diferenciarlas en el output, asigna a la primera project_name "phi-emissions-cpu" y a la segunda "phi-emissions-gpu".
Chequea en cada ejecución, que tracker._hardware es gpu y luego cpu. Tendrás que reiniciar el notebook.

Observa los resultados y analizalos.